In [33]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

Feature extracted with Mediapipe
Keypoint has x, y, z, visibility for one landmark

33 pose landmarks
21 for each hands = total 42 landmarks
468 face landmarks
Each has 3 coordinates associated

Format of data: (90, 30, 1602) -> 90 videos, 30 frames, 1662 keypoints
Lables = 90

In [34]:
import json

glosses = []
gloss_map = {}
with open('/Users/esther/Downloads/wlasl_class_list.txt', 'r') as labels:
    for line in labels:
        parts = line.strip().split('\t')
        if len(parts) == 2:
            glosses.append(parts[1])
            gloss_map[parts[1]] = int(parts[0])

glosses = np.array(glosses[:100])

In [35]:
HAND_IDENTIFIERS = [
    "wristR",
    "thumb_cmcR",
    "thumb_mcpR",
    "thumb_ipR",
    "thumb_tipR",
    "index_mcpR",
    "index_pipR",
    "index_dipR",
    "index_tipR",
    "middle_mcpR",
    "middle_pipR",
    "middle_dipR",
    "middle_tipR",
    "ring_mcpR",
    "ring_finger_pipR",
    "ring_finger_dipR",
    "ring_finger_tipR",
    "pinky_mcpR",
    "pinky_pipR",
    "pinky_dipR",
    "pinky_tipR",
    "wristL",
    "thumb_cmcL",
    "thumb_mcpL",
    "thumb_ipL",
    "thumb_tipL",
    "index_mcpL",
    "index_pipL",
    "index_dipL",
    "index_tipL",
    "middle_mcpL",
    "middle_pipL",
    "middle_dipL",
    "middle_tipL",
    "ring_mcpL",
    "ring_pipL",
    "ring_dipL",
    "ring_tipL",
    "pinky_mcpL",
    "pinky_pipL",
    "pinky_dipL",
    "pinky_tipL"
]

BODY_IDENTIFIERS = [
    "nose",
    "eye_innerL",
    "eye_innerR",
    "eye_outerL",
    "eye_outerR",
    "mouthL",
    "mouthR",
    "shoulderL",
    "shoulderR",
    "wrist_poseL",
    "wrist_poseR",
    "elbowL",
    "elbowR"
]

In [36]:
# import pandas as pd
# import ast

# train = pd.read_csv('/Users/esther/Downloads/train.csv')
# X_data = train.drop('label', axis=1)
# X_data = X_data.head(2)

# data = []

# for row_index, row in X_data.iterrows():
#     current_row = np.empty(shape=(len(ast.literal_eval(row["wristR_x"])), len(BODY_IDENTIFIERS + HAND_IDENTIFIERS), 2))
    
#     for index, identifier in enumerate(HAND_IDENTIFIERS):
#         current_row[:, index, 0] = ast.literal_eval(row[identifier + "_x"])
#         current_row[:, index, 1] = ast.literal_eval(row[identifier + "_y"])

#     current_row = current_row

#     data.append(current_row)

KeyError: 'wristR_x'

In [39]:
import pandas as pd
import ast

train = pd.read_csv('/Users/esther/Downloads/train.csv')
X_data = train.drop('label', axis=1)

data = []
max_time_steps = 200  # Define the uniform length for padding

# Assuming BODY_IDENTIFIERS and HAND_IDENTIFIERS are predefined
for row_index, row in X_data.iterrows():
    # Initialize current_row with NaNs and shape of (max_time_steps, number of features, 2)
    current_row = np.full((max_time_steps, len(BODY_IDENTIFIERS + HAND_IDENTIFIERS), 2), np.nan, dtype=np.float32)

    for index, identifier in enumerate(HAND_IDENTIFIERS):
        x_values = ast.literal_eval(row.get(identifier + "_X", '[]'))
        y_values = ast.literal_eval(row.get(identifier + "_Y", '[]'))

        actual_length = min(len(x_values), max_time_steps)

        current_row[:actual_length, index, 0] = x_values[:actual_length]
        current_row[:actual_length, index, 1] = y_values[:actual_length]

    current_row[np.isnan(current_row)] = 0
    current_row[np.isinf(current_row)] = 0

    current_row = current_row.reshape(max_time_steps, -1)

    #current_row = current_row.flatten()

    data.append(current_row)

In [40]:
import csv
import pandas as pd
from tensorflow.keras.utils import to_categorical

X_train = np.array(data)

Y_train = np.array(to_categorical(train['label']).astype(int))

# val = pd.read_csv('/Users/esther/Downloads/val.csv')
# X_val = val.drop('labels', axis=1)
# Y_val = val['labels']
# Y_val = to_categorical(Y_val).astype(int)

In [41]:
print(X_train.shape)

(1442, 200, 110)


In [42]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [43]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [44]:
model = Sequential()
# Add 64 LSTM Units - need to return sequences for the next layer
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(200,110)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(glosses.shape[0], activation='softmax'))

In [45]:
from tensorflow.keras.optimizers.legacy import Adam  # Import the legacy version

# Assuming you are compiling a model
model.compile(
    optimizer=Adam(learning_rate=0.001),  # Use the legacy Adam optimizer
    loss='categorical_crossentropy',
    metrics=['categorical_accuracy']
)

In [46]:
model.fit(X_train, Y_train, epochs=100, callbacks=[tb_callback])

Epoch 1/100
46/46 [==============================] - 11s 207ms/step - loss: 4.6061 - categorical_accuracy: 0.0118
Epoch 2/100
46/46 [==============================] - 10s 216ms/step - loss: 4.6049 - categorical_accuracy: 0.0208
Epoch 3/100
46/46 [==============================] - 10s 222ms/step - loss: 4.6041 - categorical_accuracy: 0.0208
Epoch 4/100
46/46 [==============================] - 10s 220ms/step - loss: 4.6033 - categorical_accuracy: 0.0208
Epoch 5/100
46/46 [==============================] - 10s 219ms/step - loss: 4.6026 - categorical_accuracy: 0.0208
Epoch 6/100
46/46 [==============================] - 10s 219ms/step - loss: 109305634226176.0000 - categorical_accuracy: 0.0201
Epoch 7/100
46/46 [==============================] - 10s 223ms/step - loss: nan - categorical_accuracy: 0.0208
Epoch 8/100
32/46 [===================>..........] - ETA: 3s - loss: nan - categorical_accuracy: 0.0205

KeyboardInterrupt: 

In [ ]:
model.predict(X_val)